In [1]:
import pandas as pd
import batman
import emcee as mc
import numpy as np 
import matplotlib.pyplot as plt
import corner

def model_fn(theta, hjd):
    t0, rp, a, inc, per = theta   #deparse list into separate variables
    parameters = batman.TransitParams()
    parameters.ecc = 0
    parameters.w = 0
    parameters.u = [0.22, 0.32] # limb coefficients
    parameters.limb_dark = 'quadratic' # limb darkening model
    
    parameters.t0 = t0 #mcmc variable
    parameters.rp = rp #mcmc variable
    parameters.a = a #mcmc variable
    parameters.inc = inc #mcmc variable
    parameters.per = per #mcmc variable
    
    #Generating data points
    return batman.TransitModel(parameters, hjd).light_curve(parameters)
    
def likelihood_fn(theta, hjd, flux, err):
    t0, rp, a, inc, per = theta   #deparse list into separate variables
    y_model = model_fn(theta, hjd)
    
    likelihood = 0
    for i in range(len(hjd)):
        likelihood += ((flux[i] - y_model[i]) / err[i]) ** 2
    likelihood *= - 0.5
    return likelihood


def prior_fn(theta):
    t0, rp, a, inc, per = theta
    if (2453989.7 < t0 < 2453989.8) and (2.2 < per < 2.6) and (0.12 < rp < 0.13 ) and (7.5 < a < 7.7) and (83 < inc < 85):
        return 0
    else:
        return - np.inf

def prob_fn(theta, hjd, flux, err):
    lp = prior_fn(theta)
    if not np.isfinite(lp):
        return - np.inf
    return lp + likelihood_fn(theta, hjd, flux, err)


def _p0(initial, n_dim, n_walkers):
    initial_params = [np.array(initial) + [0.001, 0.001, 0.01, 0.001, 0.001]*np.random.randn(n_dim) for i in range (n_walkers)]
    return initial_params


data =pd.read_excel(r"C:\Users\omarj\Downloads\kepler_lc_group_4.xlsx")
hjd = np.array(data.HJD)
flux = np.array(data.Rel_Flux)
err = np.array(data.Flux_err)

n_walkers = 500
initial = np.array([2453989.75, 0.13, 7.6, 83, 2.5])
n_dim = len(initial)
p0 = _p0(initial, n_dim, n_walkers)  #initial parameters

#mcmc initialisation
sampler = mc.EnsembleSampler(n_walkers, n_dim, prob_fn, args=(hjd, flux, err))

p, _, _ = sampler.run_mcmc(p0, 1000, progress=True) #initial run, 1000 iterations
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")
sampler.reset()

p, _, _ = sampler.run_mcmc(p, 4000, progress=True) # another 4k iterations. Total of 5k so far
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")
sampler.reset()

p, _, _ = sampler.run_mcmc(p, 5000, progress=True) # another 5k iterations. Total of 10k so far
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")
sampler.reset()

p, _, _ = sampler.run_mcmc(p, 10000, progress=True) # another 5k iterations. Total of 10k so far
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")
sampler.reset()

p, _, _ = sampler.run_mcmc(p, 30000, progress=True) # another 5k iterations. Total of 10k so far
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")
sampler.reset()

p, _, _ = sampler.run_mcmc(p, 50000, progress=True) # another 5k iterations. Total of 10k so far
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")

p, _, _ = sampler.run_mcmc(p, 100000, progress=True) # another 5k iterations. Total of 10k so far
samples = sampler.flatchain
print(f"t0 = {samples.T[0].mean()}, rp = {samples.T[1].mean()}, a = {samples.T[2].mean()}, inc = {samples.T[3].mean()}, per = {samples.T[4].mean()}")


# output of corner plot
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams["font.size"] = 14
plt.rcParams["font.weight"] = "normal"

fig = corner.corner(samples, show_titles=True ,labels=[fr'$t_0$', r'$\frac{r_p}{R_{\star}}$', r"$\frac{a}{R_{\star}}$", fr'$\theta$', fr'$T$'],  plot_datapoints=True, quantiles=[0,0.5,1]) #plot parameters
plt.savefig('task2_mcmc_v2', dpi=600, bbox_inches='tight')
plt.show()

# plot model function with optimised parameters
plt.figure(figsize=(15,11))
plt.minorticks_on()
plt.grid(which='major', linestyle='-')
plt.grid(which='minor', linestyle='--')
plt.title(fr"Relative Flux  against Time from cetral transit / HJD.")
plt.ylabel(r"Relative Flux")
plt.xlabel(r"Time from cetral transit / HJD")

plt.errorbar(hjd, flux, xerr=0, yerr=err, fmt='o', color='k', elinewidth=2, capsize=3, capthick=3, ecolor='gray')  #errorbars
plt.plot(hjd, flux,  'ko', label='Experimental Data')
plt.plot(hjd, model_fn([samples.T[0].mean(), samples.T[1].mean(), samples.T[2].mean(), samples.T[3].mean(), samples.T[4].mean()], hjd), label='Trendline', color='r')
plt.legend()



# computing the standard deviation with the highest likelihood model
samples = sampler.flatchain  # Retrieve MCMC samples

# Compute mean and standard deviation for each parameter from MCMC samples
param_means = np.mean(samples, axis=0)
param_std = np.std(samples, axis=0)

# Find index of highest likelihood model
likelihood_values = sampler.flatlnprobability  # Extract likelihood values ( log probability)
The_highest_likelihood = np.argmax(likelihood_values) # choosing the highest likelihood value
highest_likelihood_params = samples[The_highest_likelihood] 

# Plot posterior distribution with corner plot
fig = corner.corner(samples, labels=['T_0', 'R_p/R_*', 'a/R_*', 'i', 'P'], show_titles=True, plot_datapoints=True)


# Mark the highest likelihood model on the corner plot
ndim = len(initial)
axes = np.array(fig.axes).reshape((ndim, ndim))
for i in range(ndim):
    for j in range(ndim):
        ax = axes[i, j]
        if j < i:
            ax.plot(highest_likelihood_params[j], highest_likelihood_params[i], 'rs')  # Mark the highest likelihood model

# Compute and plot 1$\sigma$ spread on diagonal elements
for i, ax in enumerate(axes.diagonal()):
    mean = param_means[i]
    std = param_std[i]
    ax.axvline(mean, color='b', label='Mean')
    ax.axvline(mean - std, color='b', linestyle='--', label=' $\pm$1$\sigma$')
    ax.axvline(mean + std, color='b', linestyle='--')
    ax.set_yticks([])  # Remove y-axis ticks for clarity

plt.legend()
plt.savefig('posterior_with_likelihood_model.png', dpi=600, bbox_inches='tight')
plt.show()

<>:158: SyntaxWarning: invalid escape sequence '\p'
<>:158: SyntaxWarning: invalid escape sequence '\p'
C:\Users\omarj\AppData\Local\Temp\ipykernel_15288\1603806386.py:158: SyntaxWarning: invalid escape sequence '\p'
  ax.axvline(mean - std, color='b', linestyle='--', label=' $\pm$1$\sigma$')
C:\Users\omarj\AppData\Local\Temp\ipykernel_15288\1603806386.py:158: SyntaxWarning: invalid escape sequence '\p'
  ax.axvline(mean - std, color='b', linestyle='--', label=' $\pm$1$\sigma$')


ModuleNotFoundError: No module named 'batman'